# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-6981ce1fc8-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Sport Workflow Series ("F1 pilots explorative search") 

Consider the following exploratory information need:

> explore F1 pilots, their nationality, racing teams, and the country of the parent organization of the team

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P17`     | country       | predicate |
| `wd:Q10497835`| F1 Team       | node |
| `wd:Q10841764`| F1 Driver       | node |
| `wd:Q9671`    | Michael Schumacher      | node |
| `wd:Q169898`  | Scuderia Ferrari         | node |
| `wd:Q27586`   |  Ferrari         | node |
| `wd:Q183`     | Germany       | node |


Also consider

```
?p wdt:P106 wd:Q10841764 .
?p wdt:P27 wd:Q183 .
```

is the BGP to retrieve all **citizens of Germany**  (driver tedeschi)

```
?p wdt:P17 wd:Q183 .
?p wdt:P31 wd:Q10497835 .

``` 

is the BGP to retrieve all **F1 teams of Germany**

## Workload Goals

1. Identify the BGP for obtaining the team of a given  F1 driver  and also the F1 competitions in which they participated

2. Identify the BGP that connects a F1 team to a parent company

3. How many F1 drivers and F1 teams exist in each country?

4. Compare number of F1 drivers to their nationality
 
   4.1 Is there some driver that joined a team in their own country?
   
   4.2 Which team employed the larger number of drivers?
   
   4.3 For each country , which are the teams and drivers with the highest number of F1 race participations


In [1]:
# start your workflow here

In [3]:
queryString = """
SELECT (COUNT(?p) as ?numOfGermanDrivers)
WHERE { 

?p wdt:P106 wd:Q10841764 .
?p wdt:P27 wd:Q183 .

} 
GROUP BY ?num
"""

print("Results")
run_query(queryString)

Results
[('numOfGermanDrivers', '58')]


1

Osservo tutte le proprietà dei driver (tedeschi) per vedere se trovo delle info utili. Nome del team e f1 competition 

In [3]:
queryString = """
SELECT DISTINCT ?p ?pName
WHERE { 

?driver wdt:P106 wd:Q10841764 .
?driver wdt:P27 wd:Q183 ;
        ?p ?obj .
        
OPTIONAL {?p <http://schema.org/name> ?pName .}

} 
ORDER BY ASC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1006')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1741')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2163')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P227')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P244')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2671')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P268')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P269')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P349')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P434')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P5361')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646')]
[('p', 'http://schema.org/name')]
[('p', 'http://www.wikidata.org/prop/direct/P3427'), ('pName', 'AS.com athl

158

ho trovato come interessanti:

P2094 "competition class" <br>
P108 "employer" <br>
P54 "member of sport team" <br>
P1344 "partecipant in" <br>
P735 "given name" <br>

Explore the property for a german driver. First of all I search for a german driver.

In [19]:
queryString = """
SELECT DISTINCT ?driver ?dName
WHERE { 

?driver wdt:P106 wd:Q10841764 .
?driver wdt:P27 wd:Q183 .
        
OPTIONAL {?driver <http://schema.org/name> ?dName .}

} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('driver', 'http://www.wikidata.org/entity/Q49235'), ('dName', 'Adrian Sutil')]
[('driver', 'http://www.wikidata.org/entity/Q57459'), ('dName', 'Jochen Mass')]
[('driver', 'http://www.wikidata.org/entity/Q58654'), ('dName', 'Hans-Joachim Stuck')]
[('driver', 'http://www.wikidata.org/entity/Q61295'), ('dName', 'Theo Helfrich')]
[('driver', 'http://www.wikidata.org/entity/Q61541'), ('dName', 'Hans Stuck')]
[('driver', 'http://www.wikidata.org/entity/Q75810'), ('dName', 'Nick Heidfeld')]
[('driver', 'http://www.wikidata.org/entity/Q2165501'), ('dName', 'Adolf Lang')]
[('driver', 'http://www.wikidata.org/entity/Q57451'), ('dName', 'Wolfgang von Trips')]
[('driver', 'http://www.wikidata.org/entity/Q2253423'), ('dName', 'Ludwig Fischer')]
[('driver', 'http://www.wikidata.org/entity/Q42311'), ('dName', 'Sebastian Vettel')]


10

sebastian vettel "Q42311". Usin the previous properties

In [22]:
queryString = """
SELECT ?compClass ?cName ?employer ?eName ?member ?mName ?partIn ?pName
WHERE { 

wd:Q42311 wdt:P2094 ?compClass ;
            wdt:P108 ?employer ;
            wdt:P54 ?member ;
            wdt:P1344 ?partIn .
        
OPTIONAL {?compClass <http://schema.org/name> ?cName .
            ?employer <http://schema.org/name> ?eName .
            ?member <http://schema.org/name> ?mName .
            ?partIn <http://schema.org/name> ?pName .

}

} 

"""

print("Results")
run_query(queryString)

Results
Empty


0

In [7]:
#non va? 
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q42311 ?p ?obj .
        
OPTIONAL {?p <http://schema.org/name> ?pName .}
OPTIONAL{?obj <http://schema.org/name> ?objName .}

} 
ORDER BY ASC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://schema.org/name'), ('obj', 'Sebastian Vettel')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('obj', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1741'), ('obj', 'http://data.beeldengeluid.nl/gtaa/225470')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214'), ('obj', 'http://viaf.org/viaf/120235751')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2163'), ('obj', 'http://id.worldcat.org/fast/1955448')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P227'), ('obj', 'https://d-nb.info/gnd/140974741')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P244'), ('obj', 'https://id.loc.gov/authorities/names/no2012112192')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646'), ('obj', 'http://g.co/kg/m/0g5brh')]
[('p', 'http://www.wikidata.org/prop/direct/P3427'), ('pName', 'AS.com athlete ID'), ('obj', 'sebastian_vettel/24343')]
[('p', 'http://www.wikidata.org/prop/dire

89

Vettel non ha tutte le proprietà precedenti... solo alcune

"Identify the BGP for obtaining the team of a given  F1 driver  and also the F1 competitions in which they participated"

Q10841764 "F1 driver" f1 driver category

In [6]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q10841764 ?p ?obj .
        
OPTIONAL {?p <http://schema.org/name> ?pName .}
OPTIONAL{?obj <http://schema.org/name> ?objName .}
} 
ORDER BY ASC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('obj', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P8408'), ('obj', 'http://kbpedia.org/kko/rc/FormulaOneRacer')]
[('p', 'http://schema.org/name'), ('obj', 'Formula One driver')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('pName', 'KBpedia ID'), ('obj', 'FormulaOneRacer')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('pName', 'field of this occupation'), ('obj', 'http://www.wikidata.org/entity/Q1968'), ('objName', 'Formula One')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list'), ('obj', 'http://www.wikidata.org/entity/Q172853'), ('objName', 'list of Formula One drivers')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q28640'), ('objName', 'profession')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.o

10

Esploro F1 Q1968 in cerca di qualcosa legato alle edizioni

In [8]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q1968 ?p ?obj .
        
OPTIONAL {?p <http://schema.org/name> ?pName .}
OPTIONAL{?obj <http://schema.org/name> ?objName .}
} 
ORDER BY ASC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('obj', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214'), ('obj', 'http://viaf.org/viaf/245922962')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P227'), ('obj', 'https://d-nb.info/gnd/4267570-4')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P244'), ('obj', 'https://id.loc.gov/authorities/names/nb2012002495')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P268'), ('obj', 'http://data.bnf.fr/ark:/12148/cb12002946q#about')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P349'), ('obj', 'http://id.ndl.go.jp/auth/ndlna/00960053')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P5429'), ('obj', 'http://cv.iptc.org/newscodes/mediatopic/20000994')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646'), ('obj', 'http://g.co/kg/m/02xz2')]
[('p', 'http://schema.org/name'), ('obj', 'Formula One')]
[('p', 'http://www.wikidata.org/p

64

Exploring Q2705092 Formula One racing

In [10]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q2705092 ?p ?obj .
        
OPTIONAL {?p <http://schema.org/name> ?pName .}
OPTIONAL{?obj <http://schema.org/name> ?objName .}
} 
ORDER BY ASC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('obj', 'http://wikiba.se/ontology#Item')]
[('p', 'http://schema.org/name'), ('obj', 'Formula One racing')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q11924610'), ('objName', 'Formula One Grand Prix')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q11924610'), ('objName', 'Formula One Grand Prix')]
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('pName', 'practiced by'), ('obj', 'http://www.wikidata.org/entity/Q10841764'), ('objName', 'Formula One driver')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q1199515'), ('objName', 'formula racing')]


6

Ho trovato Formula 1 Grand prix Q11924610

In [11]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q11924610 ?p ?obj .
        
OPTIONAL {?p <http://schema.org/name> ?pName .}
OPTIONAL{?obj <http://schema.org/name> ?objName .}
} 
ORDER BY ASC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('obj', 'http://wikiba.se/ontology#Item')]
[('p', 'http://schema.org/name'), ('obj', 'Formula One Grand Prix')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category'), ('obj', 'Formula One Grands Prix')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q2705092'), ('objName', 'Formula One racing')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list'), ('obj', 'http://www.wikidata.org/entity/Q179371'), ('objName', 'list of Formula One Grands Prix')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q65746180'), ('objName', 'structure of the Formula One Grand Prix')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q65742449'), ('objName', 'Formula One race')]
[('p', 'http:

11

Questa è la lista dei grand prix...Q179371. Exploring cosidering gran prix as an object

In [12]:
queryString = """
SELECT ?s ?sName ?p ?pName
WHERE { 

?s ?p wd:Q11924610  .
        
OPTIONAL {?s <http://schema.org/name> ?sName .}
OPTIONAL{?p <http://schema.org/name> ?pName .}
} 
ORDER BY ASC (?sName)
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q106861620'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of')]
[('s', 'http://www.wikidata.org/entity/Q20855929'), ('p', 'http://www.wikidata.org/prop/direct/P921'), ('pName', 'main subject')]
[('s', 'http://www.wikidata.org/entity/Q21826783'), ('p', 'http://www.w3.org/2002/07/owl#sameAs')]
[('s', 'http://www.wikidata.org/entity/Q12766446'), ('p', 'http://www.w3.org/2002/07/owl#sameAs')]
[('s', 'http://www.wikidata.org/entity/Q12390157'), ('p', 'http://www.w3.org/2002/07/owl#sameAs')]
[('s', 'https://www.wikidata.org/wiki/Special:EntityData/Q11924610'), ('p', 'http://schema.org/about')]
[('s', 'http://www.wikidata.org/entity/Q67123903'), ('sName', '2020 Australian Grand Prix'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q67123905'), ('sName', '2020 Bahrain Grand Prix'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of

70

Q7849 looking for the edition of that event. 

In [13]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q7849 ?p ?obj  .
        
OPTIONAL {?obj <http://schema.org/name> ?objName .}
OPTIONAL{?p <http://schema.org/name> ?pName .}
} 
ORDER BY ASC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('obj', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646'), ('obj', 'http://g.co/kg/m/02q3d9b')]
[('p', 'http://schema.org/name'), ('obj', 'Abu Dhabi Grand Prix')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category'), ('obj', 'Abu Dhabi Grand Prix')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pName', 'Freebase ID'), ('obj', '/m/02q3d9b')]
[('p', 'http://www.wikidata.org/prop/direct/P5247'), ('pName', 'Giant Bomb ID'), ('obj', '3015-4894')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q878'), ('objName', 'United Arab Emirates')]
[('p', 'http://www.wikidata.org/prop/direct/P2257'), ('pName', 'event interval'), ('obj', '1')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q19601896'), ('objName', '2015 Abu

26

"Has part": Times a Grand Prix was held. 

Monza Q8036 ?

In [14]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q8036 ?p ?obj  .
        
OPTIONAL {?obj <http://schema.org/name> ?objName .}
OPTIONAL{?p <http://schema.org/name> ?pName .}
} 
ORDER BY ASC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('obj', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214'), ('obj', 'http://viaf.org/viaf/315594845')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P244'), ('obj', 'https://id.loc.gov/authorities/names/n2015027156')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646'), ('obj', 'http://g.co/kg/m/02k2fp')]
[('p', 'http://schema.org/name'), ('obj', 'Italian Grand Prix')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category'), ('obj', 'Italian Grand Prix')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pName', 'Freebase ID'), ('obj', '/m/02k2fp')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('pName', 'Library of Congress authority ID'), ('obj', 'n2015027156')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('pName', 'Quora topic ID'), ('obj', 'Italian-Grand-Prix')]
[('p', 'http://www.wikidata.org/prop/direct/P2

112

Yes, "has part" it is the number of times a circut was used by F1. 

NowI can search for the list of driver present in an edition. 2012 "Q173101"

In [15]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q173101 ?p ?obj  .
        
OPTIONAL {?obj <http://schema.org/name> ?objName .}
OPTIONAL{?p <http://schema.org/name> ?pName .}
} 
ORDER BY ASC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('obj', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646'), ('obj', 'http://g.co/kg/m/0ll0x90')]
[('p', 'http://schema.org/name'), ('obj', '2012 Italian Grand Prix')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category'), ('obj', '2012 Italian Grand Prix')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pName', 'Freebase ID'), ('obj', '/m/0ll0x90')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('objName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P3157'), ('pName', 'event distance'), ('obj', '53')]
[('p', 'http://www.wikidata.org/prop/direct/P5053'), ('pName', 'fastest lap'), ('obj', 'http://www.wikidata.org/entity/Q75820'), ('objName', 'Nico Rosberg')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikida

16

Provo al contrario

In [16]:
queryString = """
SELECT ?s ?sName ?p ?pName
WHERE { 

?s ?p wd:Q173101 .
        
OPTIONAL {?s <http://schema.org/name> ?sName .}
OPTIONAL{?p <http://schema.org/name> ?pName .}
} 
ORDER BY ASC (?sName)
"""

print("Results")
run_query(queryString)

Results
[('s', 'https://www.wikidata.org/wiki/Special:EntityData/Q173101'), ('p', 'http://schema.org/about')]
[('s', 'http://www.wikidata.org/entity/Q69959'), ('sName', '2012 Formula One World Championship'), ('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part')]
[('s', 'http://www.wikidata.org/entity/Q8036'), ('sName', 'Italian Grand Prix'), ('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part')]


3

Monza as an object

In [18]:
queryString = """
SELECT ?s ?sName ?p ?pName
WHERE { 

?s ?p wd:Q8036  .
        
OPTIONAL {?s <http://schema.org/name> ?sName .}
OPTIONAL{?p <http://schema.org/name> ?pName .}
} 
ORDER BY ASC (?sName)
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q56409636'), ('p', 'http://www.w3.org/2002/07/owl#sameAs')]
[('s', 'http://www.wikidata.org/entity/Q6104909'), ('p', 'http://www.w3.org/2002/07/owl#sameAs')]
[('s', 'https://www.wikidata.org/wiki/Special:EntityData/Q8036'), ('p', 'http://schema.org/about')]
[('s', 'http://www.wikidata.org/entity/Q173750'), ('sName', '1921 Italian Grand Prix'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q173748'), ('sName', '1922 Italian Grand Prix'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q173753'), ('sName', '1923 Italian Grand Prix'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q3113815'), ('sName', '1924 Italian Grand Prix'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity

95

"formula one race" entity Q65742449

In [19]:
queryString = """
SELECT ?s ?sName ?p ?pName
WHERE { 

?s ?p wd:Q65742449  .
        
OPTIONAL {?s <http://schema.org/name> ?sName .}
OPTIONAL{?p <http://schema.org/name> ?pName .}
} 
ORDER BY ASC (?sName)
"""

print("Results")
run_query(queryString)

Results
[('s', 'https://www.wikidata.org/wiki/Special:EntityData/Q65742449'), ('p', 'http://schema.org/about')]
[('s', 'http://www.wikidata.org/entity/Q49000192'), ('sName', '1951 Bordeaux Grand Prix'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q49000184'), ('sName', '1951 Richmond Trophy'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q7902163'), ('sName', 'Category:1973 Formula One races'), ('p', 'http://www.wikidata.org/prop/direct/P4224'), ('pName', 'category contains')]
[('s', 'http://www.wikidata.org/entity/Q105801254'), ('sName', 'Category:2021 Formula One races'), ('p', 'http://www.wikidata.org/prop/direct/P971'), ('pName', 'category combines topics')]
[('s', 'http://www.wikidata.org/entity/Q100251706'), ('sName', 'Eifel Grand Prix'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of')]
[('s', 'http://www.wikidata

11

In [20]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q65742449 ?p ?obj   .
        
OPTIONAL {?obj <http://schema.org/name> ?objName .}
OPTIONAL{?p <http://schema.org/name> ?pName .}
} 
ORDER BY ASC (?sName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2705092'), ('objName', 'Formula One racing')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q11924610'), ('objName', 'Formula One Grand Prix')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5386'), ('objName', 'auto racing')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q87267404'), ('objName', 'formula race')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('obj', 'http://wikiba.se/ontology#Item')]
[('p', 'http://schema.org/name'), ('obj', 'Formula One race')]


6

PUNTO 1 SDA COPLETARE---- NON TROVO RISPOSTA. PASSO AL 2, DOVE HO GIà MOLTE PIù INFORMAZIONI

-----------------------------

**Identify the BGP that connects a F1 team to a parent company**

Looking at Vettel teams I can see many possibilities. Starting from the recent one: AMR<br>
Q173663 Red Bull Racing <br>
Q750016 Aston Martin Racing <br>
Q141818 Scuderia Toro Rosso <br>
Q154118 BMW Sauber <br>
Q169898 Scuderia Ferrari <br>

In [3]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q750016 ?p ?obj   .
        
OPTIONAL {?obj <http://schema.org/name> ?objName .}
OPTIONAL{?p <http://schema.org/name> ?pName .}
} 
ORDER BY ASC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('obj', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646'), ('obj', 'http://g.co/kg/m/047v0hq')]
[('p', 'http://schema.org/name'), ('obj', 'Aston Martin Racing')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category'), ('obj', 'Aston Martin Racing')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pName', 'Freebase ID'), ('obj', '/m/047v0hq')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q145'), ('objName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pName', 'image'), ('obj', 'http://commons.wikimedia.org/wiki/Special:FilePath/Aston%20Martin%20Vantage%20GTE%20Lynn%20Silverstone%202018.jpg')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('pName', 'inception'), ('obj', '2004-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P31'),

11

Using P361 "part of".

I consider the reverse path

In [6]:
queryString = """
SELECT ?s ?sName ?p ?pName
WHERE { 

?s ?p wd:Q750016 .
        
OPTIONAL {?s <http://schema.org/name> ?sName .}
OPTIONAL{?p <http://schema.org/name> ?pName .}
} 
ORDER BY ASC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('s', 'https://www.wikidata.org/wiki/Special:EntityData/Q750016'), ('p', 'http://schema.org/about')]
[('s', 'http://www.wikidata.org/entity/Q27074'), ('sName', 'Aston Martin Lagonda'), ('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part')]
[('s', 'http://www.wikidata.org/entity/Q42311'), ('sName', 'Sebastian Vettel'), ('p', 'http://www.wikidata.org/prop/direct/P54'), ('pName', 'member of sports team')]
[('s', 'http://www.wikidata.org/entity/Q506657'), ('sName', 'Frédéric Makowiecki'), ('p', 'http://www.wikidata.org/prop/direct/P54'), ('pName', 'member of sports team')]
[('s', 'http://www.wikidata.org/entity/Q23900707'), ('sName', 'Thiim-Sørensen-Turner'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of')]
[('s', 'http://www.wikidata.org/entity/Q23657686'), ('sName', '2016 FIA World Endurance Cup for GT Manufacturers'), ('p', 'http://www.wikidata.org/prop/direct/P710'), ('pName', 'participant')]
[('s', 'http://www.wikidata.org/entity/Q21

8

Exploring RedBull Racing, using "part of" P361

In [7]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q173663 wdt:P361 ?obj   .
        
OPTIONAL {?obj <http://schema.org/name> ?objName .}
OPTIONAL{?p <http://schema.org/name> ?pName .}
} 
ORDER BY ASC (?pName)
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [8]:
queryString = """
SELECT ?obj ?objName
WHERE { 

wd:Q173663 wdt:P361 ?obj   .
        
OPTIONAL {?obj <http://schema.org/name> ?objName .}
OPTIONAL{?p <http://schema.org/name> ?pName .}
} 

"""

print("Results")
run_query(queryString)

Results
Empty


0

In [9]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q173663 ?p ?obj   .
        
OPTIONAL {?obj <http://schema.org/name> ?objName .}
OPTIONAL{?p <http://schema.org/name> ?pName .}
} 

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2705092'), ('objName', 'Formula One racing')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q145'), ('objName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P452'), ('pName', 'industry'), ('obj', 'http://www.wikidata.org/entity/Q190117'), ('objName', 'automotive industry')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5386'), ('objName', 'auto racing')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pName', 'owned by'), ('obj', 'http://www.wikidata.org/entity/Q698639'), ('objName', 'Red Bull GmbH')]
[('p', 'http://www.wikidata.org/prop/direct/P740'), ('pName', 'location of formation'), ('obj', 'http://www.wikidata.org/entity/Q204234'), ('objName', 'Milton Keynes')]
[('p', 'http://www.wikidata.org/prop/direct/P

30

I've found an interesting property: P127 "owned by". Exploring ToroRosso Q141818 with this new info

In [10]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q141818 ?p ?obj   .
        
OPTIONAL {?obj <http://schema.org/name> ?objName .}
OPTIONAL{?p <http://schema.org/name> ?pName .}
} 

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q1968'), ('objName', 'Formula One')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('objName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q10497835'), ('objName', 'Formula One team')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pName', 'replaced by'), ('obj', 'http://www.wikidata.org/entity/Q71371541'), ('objName', 'Scuderia AlphaTauri Grimm')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q8055888'), ('objName', 'Category:Scuderia Toro Rosso')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pName', 'image'), ('obj', 'http://commons.wikimedia.org/wiki/Special:FilePath/Daniel%20Ricciardo%202013%20Malaysia%20FP2%201.j

20

Alpha Tauri?

In [11]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q71371541 ?p ?obj   .
        
OPTIONAL {?obj <http://schema.org/name> ?objName .}
OPTIONAL{?p <http://schema.org/name> ?pName .}
} 

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pName', 'replaces'), ('obj', 'http://www.wikidata.org/entity/Q141818'), ('objName', 'Scuderia Toro Rosso')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pName', 'headquarters location'), ('obj', 'http://www.wikidata.org/entity/Q52981'), ('objName', 'Faenza')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('objName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q10497835'), ('objName', 'Formula One team')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q21748148'), ('objName', 'Formula One constructor')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after'), ('obj', 'http://www.wikidata.org/entity/Q60752400'), ('objName', 'AlphaTauri')]
[('p', 'http://www.wikidata.org/pro

21

ToroRosso/Aplha Tauri are indipendet companies. 

Looking for a driver Lance Stroll. But I don't see the property member of sport, like Vettel.

In [17]:
queryString = """
SELECT ?person ?pName
WHERE { 

?person wdt:P106 wd:Q10841764 .

OPTIONAL{?person <http://schema.org/name> ?pName .}
} 
ORDER BY DESC (?pName)
LIMIT 300
"""

print("Results")
run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q60837212')]
[('person', 'http://www.wikidata.org/entity/Q85854161')]
[('person', 'http://www.wikidata.org/entity/Q16140520')]
[('person', 'http://www.wikidata.org/entity/Q61000367')]
[('person', 'http://www.wikidata.org/entity/Q57451'), ('pName', 'Wolfgang von Trips')]
[('person', 'http://www.wikidata.org/entity/Q61495'), ('pName', 'Wolfgang Seidel')]
[('person', 'http://www.wikidata.org/entity/Q173296'), ('pName', 'Wilson Fittipaldi Júnior')]
[('person', 'http://www.wikidata.org/entity/Q171881'), ('pName', 'Willy Mairesse')]
[('person', 'http://www.wikidata.org/entity/Q2460525'), ('pName', 'William Ferguson')]
[('person', 'http://www.wikidata.org/entity/Q172546'), ('pName', 'William Cantrell')]
[('person', 'http://www.wikidata.org/entity/Q66177'), ('pName', 'Willi Krakau')]
[('person', 'http://www.wikidata.org/entity/Q61486'), ('pName', 'Willi Heeks')]
[('person', 'http://www.wikidata.org/entity/Q1866804'), ('pName', 'Willard Cantre

300

In [20]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q16238921 ?p ?obj .

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?obj <http://schema.org/name> ?objName .}
} 
ORDER BY DESC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2705092'), ('objName', 'Formula One racing')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5386'), ('objName', 'auto racing')]
[('p', 'http://www.wikidata.org/prop/direct/P8687'), ('pName', 'social media followers'), ('obj', '128349')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pName', 'sex or gender'), ('obj', 'http://www.wikidata.org/entity/Q6581097'), ('objName', 'male')]
[('p', 'http://www.wikidata.org/prop/direct/P3048'), ('pName', 'racing-reference.info driver ID'), ('obj', 'Lance_Stroll')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pName', 'place of birth'), ('obj', 'http://www.wikidata.org/entity/Q340'), ('objName', 'Montreal')]
[('p', 'http://www.wikidata.org/prop/direct/P856'), ('pName', 'official website'), ('obj', 'https://www.lancestroll.com')]
[('p', 'http://www.wiki

43

EXploring Racing Point, the old name of AMR

In [21]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q56245286 ?p ?obj .

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?obj <http://schema.org/name> ?objName .}
} 
ORDER BY DESC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q56265077'), ('objName', 'Template:Racing Point Force India')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q60882862'), ('objName', 'Category:Racing Point')]
[('p', 'http://www.wikidata.org/prop/direct/P8687'), ('pName', 'social media followers'), ('obj', '1003380')]
[('p', 'http://www.wikidata.org/prop/direct/P8687'), ('pName', 'social media followers'), ('obj', '154000')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pName', 'replaced by'), ('obj', 'http://www.wikidata.org/entity/Q104620704'), ('objName', 'Aston Martin F1 Team')]
[('p', 'http://www.wikidata.org/prop/direct/P856'), ('pName', 'official website'), ('obj', 'https://web.archive.org/web/20201231080631/https://www.racingpointf1.com/')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pName', '

23

Scuderia Ferrari wd:Q169898

In [22]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q169898 ?p ?obj .

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?obj <http://schema.org/name> ?objName .}
} 
ORDER BY DESC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q9256648'), ('objName', 'Category:Scuderia Ferrari')]
[('p', 'http://www.wikidata.org/prop/direct/P355'), ('pName', 'subsidiary'), ('obj', 'http://www.wikidata.org/entity/Q5445262'), ('objName', 'Ferrari Driver Academy')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5386'), ('objName', 'auto racing')]
[('p', 'http://www.wikidata.org/prop/direct/P859'), ('pName', 'sponsor'), ('obj', 'http://www.wikidata.org/entity/Q659379'), ('objName', 'Mahle GmbH')]
[('p', 'http://www.wikidata.org/prop/direct/P859'), ('pName', 'sponsor'), ('obj', 'http://www.wikidata.org/entity/Q14799'), ('objName', 'Lenovo')]
[('p', 'http://www.wikidata.org/prop/direct/P859'), ('pName', 'sponsor'), ('obj', 'http://www.wikidata.org/entity/Q691958'), ('objName', 'Hublot')]
[('p', 'http://www.wikidata.org/prop/direct/P8

42

After many queries I've found the property "parent organization" P749

I can start to produce the BGP for the second point: 

Starting from the driver that work for a team that is owned/controlled by another company. 

In [24]:
queryString = """
SELECT DISTINCT ?sportTeam ?tName ?parentC ?cName
WHERE { 

?person wdt:P106 wd:Q10841764 ;
        wdt:P54 ?sportTeam .
?sportTeam ?p ?parentC .

OPTIONAL{?sportTeam <http://schema.org/name> ?tName .}
OPTIONAL{?parentC <http://schema.org/name> ?cName .}

FILTER (?p = wdt:P749 OR wdt:P127 OR wdt:361)
} 
"""

print("Results")
run_query(queryString)

Results
[('sportTeam', 'http://www.wikidata.org/entity/Q22958306'), ('tName', 'Team ByKolles #4'), ('parentC', 'http://www.wikidata.org/entity/Q22936930'), ('cName', 'Le Mans Prototype 1')]
[('sportTeam', 'http://www.wikidata.org/entity/Q22958306'), ('tName', 'Team ByKolles #4'), ('parentC', 'http://www.wikidata.org/entity/Q23674434'), ('cName', 'ByKolles')]
[('sportTeam', 'http://www.wikidata.org/entity/Q22958306'), ('tName', 'Team ByKolles #4'), ('parentC', 'http://www.wikidata.org/entity/Q20074337'), ('cName', 'auto racing team')]
[('sportTeam', 'http://www.wikidata.org/entity/Q22958306'), ('tName', 'Team ByKolles #4'), ('parentC', 'http://www.wikidata.org/entity/Q18348695'), ('cName', '2015 FIA World Endurance Championship')]
[('sportTeam', 'http://www.wikidata.org/entity/Q2267054'), ('tName', 'Manor Motorsport'), ('parentC', 'http://www.wikidata.org/entity/Q145'), ('cName', 'United Kingdom')]
[('sportTeam', 'http://www.wikidata.org/entity/Q2267054'), ('tName', 'Manor Motorsport'),

1573

Too vague. I need to change something

In [25]:
queryString = """
SELECT DISTINCT ?sportTeam ?tName ?parentC ?cName
WHERE { 

?person wdt:P106 wd:Q10841764 ;
        wdt:P54 ?sportTeam .
?sportTeam ?p ?parentC .

OPTIONAL{?sportTeam <http://schema.org/name> ?tName .}
OPTIONAL{?parentC <http://schema.org/name> ?cName .}

FILTER (?p = wdt:P749)
} 
"""

print("Results")
run_query(queryString)

Results
[('sportTeam', 'http://www.wikidata.org/entity/Q173222'), ('tName', 'Gordini'), ('parentC', 'http://www.wikidata.org/entity/Q6686'), ('cName', 'Renault')]
[('sportTeam', 'http://www.wikidata.org/entity/Q169898'), ('tName', 'Scuderia Ferrari'), ('parentC', 'http://www.wikidata.org/entity/Q27586'), ('cName', 'Ferrari')]
[('sportTeam', 'http://www.wikidata.org/entity/Q172030'), ('tName', 'McLaren'), ('parentC', 'http://www.wikidata.org/entity/Q173146'), ('cName', 'McLaren Technology Group')]
[('sportTeam', 'http://www.wikidata.org/entity/Q23317'), ('tName', 'Audi'), ('parentC', 'http://www.wikidata.org/entity/Q156578'), ('cName', 'Volkswagen Group')]
[('sportTeam', 'http://www.wikidata.org/entity/Q40993'), ('tName', 'Porsche'), ('parentC', 'http://www.wikidata.org/entity/Q156578'), ('cName', 'Volkswagen Group')]
[('sportTeam', 'http://www.wikidata.org/entity/Q173314'), ('tName', 'Enzo Coloni Racing Car Systems'), ('parentC', 'http://www.wikidata.org/entity/Q1631454'), ('cName', 'S

6

In [35]:
queryString = """
SELECT DISTINCT ?sportTeam ?tName ?parentC ?cName
WHERE { 

?person wdt:P106 wd:Q10841764 ;
        wdt:P54 ?sportTeam .
?sportTeam ?p ?parentC ;
            wdt:P31 wd:Q10497835 . 

OPTIONAL{?sportTeam <http://schema.org/name> ?tName .}
OPTIONAL{?parentC <http://schema.org/name> ?cName .}

FILTER((?p = wdt:P127) OR (?p = wdt:749)) ##parent or owned
} 
"""

print("Results")
run_query(queryString)

Results
[('sportTeam', 'http://www.wikidata.org/entity/Q173663'), ('tName', 'Red Bull Racing'), ('parentC', 'http://www.wikidata.org/entity/Q698639'), ('cName', 'Red Bull GmbH')]
[('sportTeam', 'http://www.wikidata.org/entity/Q171337'), ('tName', 'Williams Grand Prix Engineering'), ('parentC', 'http://www.wikidata.org/entity/Q98714466'), ('cName', 'Dorilton Capital')]
[('sportTeam', 'http://www.wikidata.org/entity/Q98930155'), ('tName', 'Alpine F1 Team'), ('parentC', 'http://www.wikidata.org/entity/Q6686'), ('cName', 'Renault')]


3

Fixing: Only the team. Discarding the person connection.

In [42]:
queryString = """
SELECT DISTINCT  ?parentC ?cName
WHERE { 

?sportTeam ?p ?parentC ;
            wdt:P31 wd:Q10497835 . 

OPTIONAL{?sportTeam <http://schema.org/name> ?tName .}
OPTIONAL{?parentC <http://schema.org/name> ?cName .}

FILTER(?p=wdt:P361)
} 
"""

print("Results")
run_query(queryString)

Results
[('parentC', 'http://www.wikidata.org/entity/Q26921'), ('cName', 'Alfa Romeo')]
[('parentC', 'http://www.wikidata.org/entity/Q65961126'), ('cName', 'Honda in Formula One')]
[('parentC', 'http://www.wikidata.org/entity/Q35982'), ('cName', 'Matra')]
[('parentC', 'http://www.wikidata.org/entity/Q65962777'), ('cName', 'Toyota in Formula One')]
[('parentC', 'http://www.wikidata.org/entity/Q64506339'), ('cName', 'Sauber Motorsport')]
[('parentC', 'http://www.wikidata.org/entity/Q65960697'), ('cName', 'Alfa Romeo in Formula One')]
[('parentC', 'http://www.wikidata.org/entity/Q25412278'), ('cName', 'Lotus in Formula One')]


7

Exploring Lewis Hamilton Q9673

In [43]:
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE { 

wd:Q9673 ?p ?obj . 

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?obj <http://schema.org/name> ?objName .}

} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('pName', 'medical condition'), ('obj', 'http://www.wikidata.org/entity/Q84263196'), ('objName', 'COVID-19')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('obj', 'http://www.wikidata.org/entity/Q10349745'), ('objName', 'racing automobile driver')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('obj', 'http://www.wikidata.org/entity/Q10841764'), ('objName', 'Formula One driver')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q67311526'), ('objName', 'Obalky knih.cz')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q918187'), ('objName', 'British Formula Three Championship')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q962025'), ('obj

125

Looking into <br> 
Q172030 McLaren <br>
Q172721 Mercedes

In [44]:
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE { 

wd:Q172030 ?p ?obj . 

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?obj <http://schema.org/name> ?objName .}

} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('pName', 'founded by'), ('obj', 'http://www.wikidata.org/entity/Q173081'), ('objName', 'Bruce McLaren')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pName', 'headquarters location'), ('obj', 'http://www.wikidata.org/entity/Q646225'), ('objName', 'Woking')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q145'), ('objName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q298225'), ('objName', 'McLaren')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q21748148'), ('objName', 'Formula One constructor')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q10497835'), ('objName', 'Formula One team')]
[('p', 'http://www.wikidata.or

39

In [45]:
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE { 

wd:Q172721 ?p ?obj . 

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?obj <http://schema.org/name> ?objName .}

} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q65954812'), ('objName', 'Mercedes-Benz in Formula One')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q145'), ('objName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q65954812'), ('objName', 'Mercedes-Benz in Formula One')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q21748148'), ('objName', 'Formula One constructor')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q10497835'), ('objName', 'Formula One team')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q20074337'), ('objName', 'auto racin

27

Trying to modify the previous query for the parent company, for the second question. 

**Answer 2**

I have to use multiple choice for the key property: P749 "parent organization", P127 "owned by" or P361 "part of"

In [52]:
queryString = """
SELECT ?sportTeam ?tName ?parentC ?cName
WHERE { 

?sportTeam ?p ?parentC ;
            wdt:P31 wd:Q10497835 . 

OPTIONAL{?sportTeam <http://schema.org/name> ?tName .}
OPTIONAL{?parentC <http://schema.org/name> ?cName .}

FILTER (?p = wdt:P749 OR ?p = wdt:P127 OR ?p = wdt:P361)
} 
ORDER BY ASC (?tName)
"""

print("Results")
run_query(queryString)

Results
[('sportTeam', 'http://www.wikidata.org/entity/Q622489'), ('tName', 'Alfa Romeo'), ('parentC', 'http://www.wikidata.org/entity/Q26921'), ('cName', 'Alfa Romeo')]
[('sportTeam', 'http://www.wikidata.org/entity/Q84562599'), ('tName', 'Alfa Romeo F1 team (2019)'), ('parentC', 'http://www.wikidata.org/entity/Q65960697'), ('cName', 'Alfa Romeo in Formula One')]
[('sportTeam', 'http://www.wikidata.org/entity/Q61467970'), ('tName', 'Alfa Romeo F1 team (1950-51)'), ('parentC', 'http://www.wikidata.org/entity/Q65960697'), ('cName', 'Alfa Romeo in Formula One')]
[('sportTeam', 'http://www.wikidata.org/entity/Q98930155'), ('tName', 'Alpine F1 Team'), ('parentC', 'http://www.wikidata.org/entity/Q6686'), ('cName', 'Renault')]
[('sportTeam', 'http://www.wikidata.org/entity/Q2704273'), ('tName', 'Aston Martin F1 Team'), ('parentC', 'http://www.wikidata.org/entity/Q27074'), ('cName', 'Aston Martin Lagonda')]
[('sportTeam', 'http://www.wikidata.org/entity/Q173314'), ('tName', 'Enzo Coloni Racin

15

BGP

```
?sportTeam ?p ?parentC ;
            wdt:P31 wd:Q10497835 . 

OPTIONAL{?sportTeam <http://schema.org/name> ?tName .}
OPTIONAL{?parentC <http://schema.org/name> ?cName .}

FILTER (?p = wdt:P749 OR ?p = wdt:P127 OR ?p = wdt:P361)

```


------

**How many F1 drivers and F1 teams exist in each country?**

In [3]:
queryString = """
SELECT COUNT((?person)) AS ?num_drivers COUNT((?sportTeam)) as ?num_teams
WHERE { 

?person wdt:P106 wd:Q10841764 ;
        wdt:P54 ?sportTeam ;
        wdt:P27 ?country .
        
?sportTeam wdt:P31 wd:Q10497835 ;
            wdt:P17 ?country .

OPTIONAL{?sportTeam <http://schema.org/name> ?tName .}
OPTIONAL{?country <http://schema.org/name> ?cName .}

} 
GROUP BY (?num_drivers) (?num_teams)
"""

print("Results")
run_query(queryString)

Results
[('num_drivers', '37'), ('num_teams', '37')]


1

It is wrong...

I fix it. Here I consider the driver-team connection, but the reults are limited

In [4]:
queryString = """
SELECT COUNT(?sportTeam) as ?num_teams ?cName
WHERE { 

?person wdt:P106 wd:Q10841764 ;
        wdt:P54 ?sportTeam ;
        wdt:P27 ?country .
        
?sportTeam wdt:P31 wd:Q10497835 ;
            wdt:P17 ?country .

OPTIONAL{?sportTeam <http://schema.org/name> ?tName .}
OPTIONAL{?country <http://schema.org/name> ?cName .}

} 
GROUP BY (?num_teams) (?cName)
"""

print("Results")
run_query(queryString)

Results
[('num_teams', '1'), ('cName', 'Australia')]
[('num_teams', '9'), ('cName', 'Italy')]
[('num_teams', '26'), ('cName', 'United Kingdom')]
[('num_teams', '1'), ('cName', 'France')]


4

Using only the "team" entity I can retrieve more data. 

**Answer 3** 

Here I have the number of team in each country

In [65]:
queryString = """
SELECT (COUNT(DISTINCT (?sportTeam)) as ?num_teams) ?cName
WHERE { 

?sportTeam wdt:P31 wd:Q10497835 ;
            wdt:P17 ?country .

OPTIONAL{?sportTeam <http://schema.org/name> ?tName .}
OPTIONAL{?country <http://schema.org/name> ?cName .}

} 
GROUP BY (?num_teams) (?cName)
ORDER BY ASC (?num_teams)
"""

print("Results")
run_query(queryString)

Results
[('num_teams', '1'), ('cName', 'Malaysia')]
[('num_teams', '1'), ('cName', 'Spain')]
[('num_teams', '1'), ('cName', 'Australia')]
[('num_teams', '1'), ('cName', 'Switzerland')]
[('num_teams', '2'), ('cName', 'Netherlands')]
[('num_teams', '2'), ('cName', 'Japan')]
[('num_teams', '2'), ('cName', 'Russia')]
[('num_teams', '2'), ('cName', 'Germany')]
[('num_teams', '6'), ('cName', 'France')]
[('num_teams', '16'), ('cName', 'Italy')]
[('num_teams', '26'), ('cName', 'United Kingdom')]


11

In [67]:
queryString = """
SELECT (COUNT(DISTINCT (?person)) as ?num_driver) ?cName
WHERE { 


?person wdt:P106 wd:Q10841764 ;
        wdt:P54 ?sportTeam ;
        wdt:P27 ?country .
        
?sportTeam wdt:P31 wd:Q10497835 ;
            wdt:P17 ?country .

OPTIONAL{?person <http://schema.org/name> ?pName .}
OPTIONAL{?country <http://schema.org/name> ?cName .}

} 
GROUP BY (?num_driver) (?cName)
ORDER BY ASC (?num_driver)
"""

print("Results")
run_query(queryString)

Results
[('num_driver', '1'), ('cName', 'Australia')]
[('num_driver', '1'), ('cName', 'France')]
[('num_driver', '6'), ('cName', 'Italy')]
[('num_driver', '20'), ('cName', 'United Kingdom')]


4

**Answer 3**

Not all paths are preset (ex Lance Stroll query ). I remove the property P54 "member of sport team"

Using only the "driver" entity.

Here I have the number of drivers for each country. 

In [68]:
queryString = """
SELECT (COUNT(DISTINCT (?person)) as ?num_driver) ?cName
WHERE { 

?person wdt:P106 wd:Q10841764 ;
        wdt:P27 ?country .
        
OPTIONAL{?person <http://schema.org/name> ?pName .}
OPTIONAL{?country <http://schema.org/name> ?cName .}

} 
GROUP BY (?num_driver) (?cName)
ORDER BY ASC (?num_driver)
"""

print("Results")
run_query(queryString)

Results
[('num_driver', '1'), ('cName', 'Wales')]
[('num_driver', '1'), ('cName', 'Georgia')]
[('num_driver', '1'), ('cName', 'Japan')]
[('num_driver', '1'), ('cName', 'Romania')]
[('num_driver', '1'), ('cName', 'Northern Ireland')]
[('num_driver', '1'), ('cName', 'Monaco')]
[('num_driver', '1'), ('cName', 'New Zealand')]
[('num_driver', '1'), ('cName', 'West Germany')]
[('num_driver', '1'), ('cName', 'Venezuela')]
[('num_driver', '2'), ('cName', 'Poland')]
[('num_driver', '2'), ('cName', 'Denmark')]
[('num_driver', '2'), ('cName', 'Thailand')]
[('num_driver', '2'), ('cName', 'Switzerland')]
[('num_driver', '3'), ('cName', 'Australia')]
[('num_driver', '3'), ('cName', 'German Democratic Republic')]
[('num_driver', '5'), ('cName', 'Portugal')]
[('num_driver', '5'), ('cName', 'Mexico')]
[('num_driver', '6'), ('cName', 'Brazil')]
[('num_driver', '7'), ('cName', 'Ireland')]
[('num_driver', '9'), ('cName', 'Kingdom of Italy')]
[('num_driver', '9'), ('cName', 'Finland')]
[('num_driver', '14'

34

-------------

**4 - Compare the number of F1 drivers to their nationality**

 4.1 Is there some driver that joined a team in their own country?
   
 4.2 Which team employed the larger number of drivers?
   
 4.3 For each country , which are the teams and drivers with the highest number of F1 race participations


**Answer 4.1**

I print the driver, the team and the country, filtering for the country of the driver to be the same as the one of the sport team

In [7]:
queryString = """
SELECT (?pName AS ?Name) (?tName1 AS ?Team) (?cName1 AS ?Country)
WHERE { 

?person wdt:P106 wd:Q10841764 ;
        wdt:P27 ?country1 ;
        wdt:P54 ?sportTeam1 .
        
     { #sub
         SELECT ?cName ?tName WHERE
     {
             ?sportTeam wdt:P31 wd:Q10497835 ;
                        wdt:P17 ?country .
            ?sportTeam <http://schema.org/name> ?tName .
            ?country <http://schema.org/name> ?cName .
     }
     } #sub


?sportTeam1 <http://schema.org/name> ?tName1 .
?person <http://schema.org/name> ?pName .
?country1 <http://schema.org/name> ?cName1 .

FILTER(?cName=?cName1 AND ?tName = ?tName1)

} 
ORDER BY ASC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('Name', 'Alexander Albon'), ('Team', 'Red Bull Racing'), ('Country', 'United Kingdom')]
[('Name', 'Antonio Giovinazzi'), ('Team', 'Alfa Romeo'), ('Country', 'Italy')]
[('Name', 'Arturo Merzario'), ('Team', 'Scuderia Ferrari'), ('Country', 'Italy')]
[('Name', 'Bruno Giacomelli'), ('Team', 'Alfa Romeo'), ('Country', 'Italy')]
[('Name', 'Bruno Giacomelli'), ('Team', 'Life Racing Engines'), ('Country', 'Italy')]
[('Name', 'Chris Irwin'), ('Team', 'Brabham'), ('Country', 'United Kingdom')]
[('Name', 'Dave Charlton'), ('Team', 'Brabham'), ('Country', 'United Kingdom')]
[('Name', 'David Coulthard'), ('Team', 'Williams Grand Prix Engineering'), ('Country', 'United Kingdom')]
[('Name', 'David Coulthard'), ('Team', 'McLaren'), ('Country', 'United Kingdom')]
[('Name', 'David Coulthard'), ('Team', 'Red Bull Racing'), ('Country', 'United Kingdom')]
[('Name', 'Enrico Bertaggia'), ('Team', 'Andrea Moda Formula'), ('Country', 'Italy')]
[('Name', 'Enrico Bertaggia'), ('Team', 'Forti'), ('Coun

37

**Answer 4.2**

Modifying a bit one of the previous queries

In [13]:
queryString = """
SELECT (COUNT(?person) AS ?num_drivers) (?tName AS ?Team)
WHERE { 

?person wdt:P106 wd:Q10841764 ;
        wdt:P54 ?sportTeam .

?sportTeam <http://schema.org/name> ?tName .
?person <http://schema.org/name> ?pName .

} 
GROUP BY (?num_drivers) (?tName)
ORDER BY DESC (?num_drivers)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('num_drivers', '17'), ('Team', 'Team Lotus')]


1

**4.3**
Unfortuately I'm not able to answer to this point because in the previous points I did not find the path for the partecipation of a driver to the competitions...

In [ ]:
queryString = """
SELECT ?sportTeam ?tName ?parentC ?cName
WHERE { 

?person wdt:P106 wd:Q10841764 .
?person wdt:P27 wd:Q183 .

# BGP to retrieve all citizens of Germany (driver tedeschi)

?sportTeam wdt:P17 wd:Q183 .
?sportTeam wdt:P31 wd:Q10497835 .

# BGP to retrieve all F1 teams of Germany

?person wdt:P106 wd:Q10841764 ;
        wdt:P54 ?sportTeam .
?sportTeam wdt:P31 wd:Q10497835 . 

OPTIONAL{?sportTeam <http://schema.org/name> ?tName .}
OPTIONAL{?parentC <http://schema.org/name> ?cName .}

FILTER (?p = wdt:P749 OR ?p = wdt:P127 OR ?p = wdt:P361)
} 
ORDER BY ASC (?tName)
"""

print("Results")
run_query(queryString)